In [0]:
# boilerplate

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import *

# create spark session
# /Workspace/Users/datakraft867@gmail.com/books.csv
spark = SparkSession.builder.appName("books_query").getOrCreate()
books_df = spark.read.csv("/Volumes/datakraft_batch1/default/datasets/books.csv", header=True, inferSchema=True)


In [0]:
# count the number of rows
books_df.count()



11127

In [0]:
books_df.show(10)

+---------+--------------------+--------------+------+----------+-------------+-------------+----------------+---------------+-------------+------------------+--------------------+
|num_pages|             authors|average_rating|bookID|      isbn|       isbn13|language_code|publication_date|      publisher|ratings_count|text_reviews_count|               title|
+---------+--------------------+--------------+------+----------+-------------+-------------+----------------+---------------+-------------+------------------+--------------------+
|      652|J.K. Rowling/Mary...|          4.57|     1|0439785960|9780439785969|          eng|       9/16/2006|Scholastic Inc.|      2095690|             27591|Harry Potter and ...|
|      870|J.K. Rowling/Mary...|          4.49|     2|0439358078|9780439358071|          eng|        9/1/2004|Scholastic Inc.|      2153167|             29221|Harry Potter and ...|
|      352|        J.K. Rowling|          4.42|     4|0439554896|9780439554893|          eng|  

In [0]:
# create temp value
books_df.createOrReplaceTempView("books_tbl")


In [0]:
# How many total books are present in the database?
books_cnt = spark.sql(\
    f"""
    SELECT COUNT(isbn) as books_cnt FROM books_tbl
    """).show()


+---------+
|books_cnt|
+---------+
|    11127|
+---------+



In [0]:
# what is the total page count of all books published by Bill Bryson?
# cast num_pages to integer
books_df = books_df.withColumn("num_pages", col("num_pages").cast("integer"))
# verify
books_df.printSchema()
# sql
tot_pg_bb = spark.sql(\
    f"""
    SELECT SUM(num_pages)
    FROM books_tbl
    WHERE authors = "Bill Bryson"
    """).show()


root
 |-- num_pages: integer (nullable = true)
 |-- authors: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- bookID: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- publication_date: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- ratings_count: integer (nullable = true)
 |-- text_reviews_count: integer (nullable = true)
 |-- title: string (nullable = true)

+--------------+
|sum(num_pages)|
+--------------+
|        6865.0|
+--------------+



In [0]:
# for books published in 2021, what is the average book rating?
# extract year from year from publication date
avg_rating = spark.sql(\
  f"""
  WITH CTE AS (
  SELECT 
  CAST(RIGHT(publication_date, 4) AS INT) AS yr,
  ROUND(AVG(average_rating),2) AS avg_rating
  FROM books_tbl
  GROUP BY 1
  ORDER BY 1
  )

  SELECT yr, avg_rating FROM CTE WHERE yr = 1943;
  """).show()

+----+----------+
|  yr|avg_rating|
+----+----------+
|1943|      4.21|
+----+----------+



In [0]:
# for books published in 2021, what is the average book rating?
# extract year frpm year from publication date
avg_rating = spark.sql(\
  f"""
  WITH CTE AS (
  SELECT 
  CAST(RIGHT(publication_date, 4) AS INT) AS yr,
  ROUND(AVG(average_rating),2) AS avg_rating
  FROM books_tbl
  GROUP BY 1
  ORDER BY 1
  )

  SELECT yr, avg_rating FROM CTE;
  """).show()

+----+----------+
|  yr|avg_rating|
+----+----------+
|   0|      NULL|
|   4|      NULL|
|   6|      NULL|
|1900|      3.88|
|1913|      3.96|
|1914|      3.95|
|1919|      4.32|
|1921|      4.14|
|1922|       5.0|
|1923|      4.29|
|1925|      3.96|
|1928|      4.34|
|1929|      4.02|
|1931|      2.75|
|1935|      3.63|
|1940|      3.97|
|1943|      4.21|
|1947|      4.06|
|1948|      3.55|
|1949|       3.8|
+----+----------+
only showing top 20 rows



In [0]:
publisher = spark.sql(\
    f"""
    SELECT DISTINCT publisher
    FROM books_tbl
    WHERE publisher like '%Oxford University Press%'
    """).display()

publisher
Oxford University Press
Oxford University Press USA


In [0]:
# Oxford University Press and published in 2002, list the books? 

publisher = spark.sql(
    """
    SELECT DISTINCT publisher, title, CAST(RIGHT(publication_date, 4) AS INT) AS published_year
    FROM books_tbl
    WHERE publisher LIKE 'Oxford University Press' AND CAST(RIGHT(publication_date, 4) AS INT) = 1988
    ORDER BY publisher
    """
).display()

publisher,title,published_year


In [0]:
#Which publisher had the books with the highest rating; # publisher, book_title, rating
rating = spark.sql(
    """
SELECT publisher, max(title) as book_title , max(average_rating) AS rating
FROM books_tbl
GROUP BY publisher
 """
).display()


publisher,book_title,rating
1/15/1999,Patriots (The Coming Collapse),Rawles
10/18,La mezzanine,3.84
1st Book Library,Lila's Child: An Inquiry into Quality,3.58
1st World Library,Nineteen Eighty-Four,4.18
4/20/2004,Streetcar Suburbs: The Process of Growth in Boston 1870-1900,Jr./Sam B. Warner
4/6/2004,The Tolkien Fan's Medieval Reader,one of the founding members of this Tolkien website)/Verlyn Flieger/Turgon (=David E. Smith)
5/1/1977,Brown's Star Atlas: Showing All The Bright Stars With Full Instructions How To Find And Use Them For Navigational Purposes And Department Of Trade Examinations.,Son & Ferguson
A & C Black (Childrens books),The Custard Kid (Black Cats),4.40
A Harvest Book/Harcourt Inc.,Women and Writing,4.07
A K PETERS,Advanced Global Illumination,4.50


In [0]:
# Oxford University Press and published in 1988, list the books?; Output --> publisher, year, book_title
# Which publisher had the books with the highest rating; # publisher, book_title, rating
# How many books did Dan Brown publish? # all books published by Dan Brown (even as co-author) # author, book_title, publication_date, 
# which book had highest text_reviews_count? # book_tile, text_reviews_count


publisher = spark.sql(\
    f"""
    SELECT DISTINCT authors
    FROM books_tbl
    WHERE authors like '%Dan Brown%'
    """).display()

authors
Dan Brown/Paul Michael
Dan Brown/Raúl Amundaray
Dan Brown/Daniel Roche
Dan Brown/Eduardo García Murillo
Dan Brown
Dan Brown/Juanjo Estrella


In [0]:
# How many books did Dan Brown publish? # all books published by Dan Brown (even as co-author) # author, book_title, publication_date

Dan_Brown = spark.sql(
    """
SELECT authors,title as book_title , publication_date
FROM books_tbl
WHERE authors LIKE 'Dan Brown' or publisher = 'Dan Brown'
 """
).display()


authors,book_title,publication_date
Dan Brown,Angels & Demons (Robert Langdon #1),4/1/2006
Dan Brown,Ángeles y demonios (Robert Langdon #1),12/1/2005
Dan Brown,The Da Vinci Code (Robert Langdon #2),3/28/2006
Dan Brown,The Da Vinci Code,3/28/2006
Dan Brown,Deception Point,4/1/2006
Dan Brown,Deception Point,5/1/2004
Dan Brown,Deception Point,8/1/2005
Dan Brown,The Da Vinci Code (Robert Langdon #2),3/28/2006
Dan Brown,Angels & Demons (Robert Langdon #1),5/3/2005
Dan Brown,The Da Vinci Code,3/28/2006


In [0]:
# which book had highest text_reviews_count? # book_tile, text_reviews_count
text_reviews = spark.sql("""
    SELECT title as book_title, text_reviews_count
    FROM books_tbl
    WHERE text_reviews_count = (
        SELECT MAX(text_reviews_count)
        FROM books_tbl
    )
""").display()

book_title,text_reviews_count
Twilight (Twilight #1),94265


In [0]:
# # How many books did each author release every year? Please note that the authors field may contain multiple authors. Output should include author, published_year_cnt

avg_rating = spark.sql(\
  f"""
  WITH CTE AS (
  SELECT 
  CAST(RIGHT(publication_date, 4) AS INT) AS yr,
  ROUND(AVG(average_rating),2) AS avg_rating
  FROM books_tbl
  GROUP BY 1
  ORDER BY 1
  )

  SELECT yr, avg_rating FROM CTE WHERE yr = 1943;
  """).show()


+----+----------+
|  yr|avg_rating|
+----+----------+
|1943|      4.21|
+----+----------+



In [0]:
## Oxford University Press and published in 1988, list the books?; 

title = spark.sql(\
f"""
WITH CTE AS (
SELECT publisher, title,
CAST(RIGHT(publication_date, 4) AS INT) AS yr
FROM books_tbl
)
SELECT * FROM CTE WHERE yr = 1998 and publisher like '%Oxford University Press%';
""").display()


publisher,title,yr
Oxford University Press,War and Peace,1998
Oxford University Press,The Histories,1998
Oxford University Press,Antigone; Oedipus the Kingn; Electra,1998
Oxford University Press USA,Moby Dick: or The White Whale (Oxford Illustrated Classics),1998
Oxford University Press,The Bostonians,1998
Oxford University Press,The Canterbury Tales,1998
Oxford University Press,Waverley,1998
Oxford University Press,The Karamazov Brothers,1998
Oxford University Press,The Castle of Otranto,1998
Oxford University Press USA,Faust Part One,1998
